<a href="https://colab.research.google.com/github/DiegoSReco/CasualInference_New/blob/main/script_ejemplo_doblelasso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
!pip install multiprocess
!pip install pyreadr
!git clone https://github.com/maxhuppertz/hdmpy.git
!pip install wget # for data loading

fatal: destination path 'hdmpy' already exists and is not an empty directory.


In [122]:
!rm -rf hdmpy
!git clone https://github.com/maxhuppertz/hdmpy.git
!pip install hdmpy


Cloning into 'hdmpy'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 89 (delta 47), reused 63 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 28.88 KiB | 5.78 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [123]:
import sys
import wget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pyreadr import read_r
import hdmpy
warnings.simplefilter('ignore')


# Obtenemos datos del repositorio del libro ***[Causal ML](https://https://causalml-book.org/)***
El data set que utilizó Barro y Lee es una muestra de $90$ países y de $63$ variables.


In [124]:
path = "https://github.com/CausalAIBook/MetricsMLNotebooks/blob/main/data/GrowthData.rda?raw=true"
barronlee = read_r(wget.download(path))
#Cargamos dataset
df_barronlee = barronlee['GrowthData']
#Dimensiones
df_barronlee.shape

(90, 63)

In [125]:
import statsmodels.api as sm
y = df_barronlee['Outcome']
W = df_barronlee.drop(['Outcome', 'intercept', 'gdpsh465'], axis=1)
D = df_barronlee['gdpsh465']
X = pd.concat([W, D], axis=1)


## **Estimación No Ortogonal**
En un enfoque **"ingenuo" (*naive*)**, usaríamos una sola regresión *Lasso* para predecir Y usando D y W, y nos enfocaríamos en el coeficiente de D. Se aplica un Lasso directamente para predecir Y usando tanto el tratamiento D como los controles W. Luego, se realiza una regresión final por OLS usando solo las variables seleccionadas por el primer paso (Lasso).


In [126]:
#Realizamos la regresión Lasso para selección de variables ----
modelo_lasso_est = hdmpy.rlasso(X, y, post=True)
coef_array = modelo_lasso_est.est['coefficients'].iloc[1:-1, :].to_numpy()
#Variables seleccionadas ----
var_id = np.where(coef_array != 0)[0]
W_sel = W.iloc[:, var_id] if len(var_id) > 0 else pd.DataFrame()
#Variables seleccionadas por Lasso y regresor objetivo
X_d = pd.concat([W_sel, D], axis=1)

In [150]:
#Realizamos la estimación por OLS
X_d_c = sm.add_constant(X_d)
reg_naive = sm.OLS(y,X_d_c ).fit(cov_type='HC1')

# Regresor objetivo (D)
est_naive = reg_naive.params['gdpsh465']
# HC1 std. error
std_naive = reg_naive.HC1_se['gdpsh465']
# 95% Intervalo de confianza
lower_ci, upper_ci = reg_naive.conf_int(alpha=0.05).loc['gdpsh465'].values
tabla_naive = pd.DataFrame(columns=["Estimador", "Error estándar", "IC inferior", "IC superior"])
tabla_naive.loc['Método naive'] = [est_naive, std_naive, lower_ci , upper_ci]
tabla_naive

,Estimador,Error estándar,IC inferior,IC superior
Método naive,-0.011268,0.005032,-0.021131,-0.001406


# **Estimación Ortogonal**

El método de Double Lasso (o Doble ML) aplica la ortogonalidad mediante el ***"partialling-out"***:


*   Paso 1: Usar Lasso para predecir el crecimiento (Y) con los controles (W) y obtener los residuos (Y~).
*   Paso 2: Usar Lasso para predecir la riqueza inicial (D) con los controles (W) y obtener sus residuos (D~).
*   Paso 3: Regresar Y~ sobre D~ para obtener el coeficiente ortogonal α.




In [129]:
#Paso 1: Usar Lasso para predecir el crecimiento (Y) con los controles (W) y obtener los residuos (Y~).
res_y = hdmpy.rlasso(W, y, post=False).est['residuals']
#Paso 2: Usar Lasso para predecir la riqueza inicial (D) con los controles (W) y obtener sus residuos (D~).
res_D = hdmpy.rlasso(W, D, post=False).est['residuals']

r_y = pd.DataFrame(res_y, columns=['r_y'])
r_D = pd.DataFrame(res_D, columns=['r_D'])

In [144]:
#Paso 3:Regresar Y~ sobre D~ para obtener el coeficiente ortogonal α.
reg_ols_DL = sm.OLS(r_y, r_D).fit(cov_type='HC1')
#Creamos tabla
est_DL = reg_ols_DL.params['r_D']
std_DL = reg_ols_DL.HC1_se['r_D']
lower_ci, upper_ci = reg_ols_DL.conf_int(alpha=0.05).loc['r_D'].values
tabla_DL = pd.DataFrame(columns=["Estimador", "Error estándar", "IC inferior", "IC superior"])
tabla_DL.loc['Método Double Lasso'] = [est_DL, std_DL, lower_ci, upper_ci]



In [145]:
#Comparación de resultados
summary = pd.concat([tabla_naivy, tabla_DL])

,Estimador,Error estándar,IC inferior,IC superior
Método naivy,-0.011268,0.005032,-0.021131,-0.001406
Método Double Lasso,-0.044693,0.017923,-0.079821,-0.009564


from matplotlib import pyplot as plt
summary['Estimador'].plot(kind='hist', bins=20, title='Estimador')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
summary['Error estándar'].plot(kind='hist', bins=20, title='Error estándar')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
summary['IC inferior'].plot(kind='hist', bins=20, title='IC inferior')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
summary['IC superior'].plot(kind='hist', bins=20, title='IC superior')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Error estándar']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Error estándar'}, axis=1)
              .sort_values('Error estándar', ascending=True))
  xs = counted['Error estándar']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = summary.sort_values('Error estándar', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Error estándar')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
summary['Estimador'].plot(kind='line', figsize=(8, 4), title='Estimador')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
summary['Error estándar'].plot(kind='line', figsize=(8, 4), title='Error estándar')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
summary['IC inferior'].plot(kind='line', figsize=(8, 4), title='IC inferior')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
summary['IC superior'].plot(kind='line', figsize=(8, 4), title='IC superior')
plt.gca().spines[['top', 'right']].set_visible(False)